# Notebook to build a netcdf file for SMSpp starting by a pypsa model

In [ ]:
folder_builder = "../data/SMSpp/UCBlockSolver"
path_smspp_pypsa = folder_builder + "/pypsa2smspp.nc4"

renewable_carriers = ["pv", "wind"]

In [ ]:
import pypsa
from helpers import build_microgrid_model
import netCDF4 as nc
import pandas as pd
import numpy as np
import re

NC_DOUBLE = "f8"
NP_DOUBLE = np.float64
NC_UINT = "u4"
NP_UINT = np.uint32

In [ ]:
n_snapshots = 1*24

buses_demand = [0]
bus_PV = None #0 #None
bus_wind = None #0
bus_storage = None
bus_hydro = None
bus_diesel = 0

n = build_microgrid_model(
    n_snapshots = n_snapshots,
    buses_demand = buses_demand,
    bus_PV = bus_PV,
    bus_wind = bus_wind,
    bus_storage = bus_storage,
    bus_diesel = bus_diesel,
    bus_hydro=bus_hydro,
    x = 10.389754,
    y = 43.720810,
    hydro_factor=0.5,
)

# n.storage_units.capital_cost /= 2
# n.storage_units.p_min_pu = 0.
# n.storage_units.p_nom_max = 10
# n.storage_units.max_hours = 0.5

n.snapshot_weightings["stores"] = 1.0

# n.generators.p_nom = 100

In [ ]:
n.snapshot_weightings

In [ ]:
n.optimize(solver_name="gurobi")

Get objective function without constant term

In [ ]:
n.generators_t.p.diesel.sum() * n.generators.marginal_cost * n.snapshot_weightings.objective.iloc[0]

In [ ]:
n.storage_units

In [ ]:
def get_bus_idx(n, bus_series, dtype="uint32"):
    """
    Returns the numeric index of the bus in the network n for each element of the bus_series.
    """
    return bus_series.map(n.buses.index.get_loc).astype(dtype)

## Prepare data for SMSpp UC blocks creation

In [ ]:
n.generators_t.p.mul(n.snapshot_weightings.objective).sum().mul(n.generators.marginal_cost).sum()

In [ ]:
marg_cost = pd.concat(
    [
        n.generators_t.p.mul(n.snapshot_weightings.objective, axis=0).sum().mul(n.generators.marginal_cost),
        n.storage_units_t.p.mul(n.snapshot_weightings.objective, axis=0).sum().mul(n.storage_units.marginal_cost),
        n.links_t.p0.mul(n.snapshot_weightings.objective, axis=0).sum().mul(n.links.marginal_cost),
        n.stores_t.p.mul(n.snapshot_weightings.objective, axis=0).sum().mul(n.stores.marginal_cost),
    ],
)
marg_cost

In [ ]:
cap_cost = pd.concat(
    [
        n.generators.eval("p_nom_opt * capital_cost"),
        n.storage_units.eval("p_nom_opt * capital_cost"),
        n.links.eval("p_nom_opt * capital_cost"),
        n.lines.eval("s_nom_opt * capital_cost"),
        n.stores.eval("e_nom_opt * capital_cost"),
    ]
)
cap_cost

In [ ]:
if hasattr(n, 'objective_constant'):
    print(n.objective + n.objective_constant)
else:
    print(n.objective)

In [ ]:
n.objective

In [ ]:
cap_cost.sum() + marg_cost.sum()

In [ ]:
marg_cost.sum()

In [ ]:
# ds.close()

## Create UCBlocks for SMSpp

In [ ]:
ds = nc.Dataset(path_smspp_pypsa, "w")


ds.setncattr("SMS++_file_type", 1)  # Set file type to 1 for problem file

# Create UCBlock

master = ds.createGroup("Block_0")  # Create the first main block

# master.id = "0"  # mandatory attribute for all blocks
master.type = "UCBlock"  # mandatory attribute for all blocks

## Create Network Data

In [ ]:
ndg = master.createGroup("NetworkData")
ndg.createDimension("NumberNodes", 1)

## Fill UCBlock

In [ ]:
### Create dimensions

n_timesteps = len(n.snapshots)
master.createDimension("TimeHorizon", n_timesteps)  # Time horizon

n_units = len(n.generators) + len(n.storage_units)  # excluding links for now
# TODO: links are not supported yet
master.createDimension("NumberUnits", n_units)  # Number of nodes
master.createDimension("NumberElectricalGenerators", n_units)  # Number of elec. units
# some unitblocks, do have multiple units inside [e.g. cascading hydro or CCGT], not considered now

# # number of thermal nodes
thermal_generators = n.generators[~n.generators.index.isin(renewable_carriers)]
n_thermal = len(thermal_generators)

# n_nodes = len(n.buses)
# master.createDimension("NumberNodes", n_nodes)  # Number of nodes

if len(n.lines) > 0:   
    master.createDimension("NumberLines", len(n.lines))  # Number of lines

# TODO: no NetworkBlock considered: to check if it is necessary to add it


### Create variables

# demand by node
loads_t = n.loads_t.p_set
active_demand = master.createVariable("ActivePowerDemand", NC_DOUBLE, ("TimeHorizon",)) #("NumberNodes", "TimeHorizon"))
active_demand[:] = loads_t.values.transpose()  # indexing between python and SMSpp is different: transpose

# Lines

if len(n.lines) > 0:

    # generators' node
    all_generators = [bus_PV, bus_wind, bus_storage, bus_hydro, bus_diesel]
    all_generators = [x for x in all_generators if x is not None]
    generator_node = master.createVariable("GeneratorNode", NC_UINT, ("NumberElectricalGenerators",))
    generator_node[:] = np.array(all_generators, dtype=NP_UINT)

    # start lines
    start_line = master.createVariable("StartLine", NC_UINT, ("NumberLines",))
    start_line[:] = get_bus_idx(n, n.lines.bus0).values
    # end lines
    end_line = master.createVariable("EndLine", NC_UINT, ("NumberLines",))
    end_line[:] = get_bus_idx(n, n.lines.bus1).values
    # Min power flow
    min_power_flow = master.createVariable("MinPowerFlow", NC_DOUBLE, ("NumberLines",))
    min_power_flow[:] = - n.lines.s_nom.values
    # Max power flow
    max_power_flow = master.createVariable("MaxPowerFlow", NC_DOUBLE, ("NumberLines",))
    max_power_flow[:] = n.lines.s_nom.values
    # Susceptance
    susceptance = master.createVariable("Susceptance", NC_DOUBLE, ("NumberLines",))
    susceptance[:] = 1 / n.lines.x.values


## Create a ThermalUnitBlock

In [ ]:
id_thermal = 0

min_power_pypsa = thermal_generators.eval("p_nom_opt * p_min_pu")
max_power_pypsa = thermal_generators.eval("p_nom_opt * p_max_pu")
linear_term_pypsa = thermal_generators.marginal_cost

for (idx_name, row) in thermal_generators.iterrows():

    tub = master.createGroup(f"UnitBlock_{id_thermal}")
    tub.id = str(id_thermal)
    tub.type = "ThermalUnitBlock"

    # Create variables


    # MinPower
    min_power = tub.createVariable("MinPower", NC_DOUBLE) #, ("TimeHorizon",))
    min_power[:] = min_power_pypsa.loc[idx_name]

    # MaxPower
    max_power = tub.createVariable("MaxPower", NC_DOUBLE) #, ("TimeHorizon",))
    max_power[:] = max_power_pypsa.loc[idx_name]

    # StartUpCost
    start_up_cost = tub.createVariable("StartUpCost", NC_DOUBLE)
    start_up_cost[:] = 0.0

    # LinearTerm
    linear_term = tub.createVariable("LinearTerm", NC_DOUBLE, ("TimeHorizon",))
    linear_term[:] = linear_term_pypsa.loc[idx_name] * n.snapshot_weightings.objective.values

    # ConstantTerm
    constant_term = tub.createVariable("ConstantTerm", NC_DOUBLE)
    constant_term[:] = 0.0

    # MinUpTime
    min_up_time = tub.createVariable("MinUpTime", NC_DOUBLE)
    min_up_time[:] = 0.0

    # MinDownTime
    min_down_time = tub.createVariable("MinDownTime", NC_DOUBLE)
    min_down_time[:] = 0.0

    # InitialPower
    initial_power = tub.createVariable("InitialPower", NC_DOUBLE)
    initial_power[:] = n.loads_t.p_set.iloc[0, id_thermal]

    # InitUpDownTime
    init_up_down_time = tub.createVariable("InitUpDownTime", NC_DOUBLE)
    init_up_down_time[:] = 1.0

    # InertiaCommitment
    inertia_commitment = tub.createVariable("InertiaCommitment", NC_DOUBLE)
    inertia_commitment[:] = 1.0

    id_thermal += 1

#### Add renewable sources

In [ ]:
renewable_generators = n.generators[n.generators.index.isin(renewable_carriers)]

id_ren = id_thermal

if not renewable_generators.empty:
    for (idx_name, row) in renewable_generators.iterrows():

        tiub = master.createGroup(f"UnitBlock_{id_ren}")
        tiub.id = str(id_ren)
        tiub.type = "IntermittentUnitBlock"

        n_max_power = n.generators_t.p_max_pu.loc[:, idx_name] * row.p_nom_opt
        

        # max power
        max_power = tiub.createVariable("MaxPower", NC_DOUBLE, ("TimeHorizon",))
        max_power[:] = n_max_power

        # # max capacity
        # max_capacity = tiub.createVariable("MaxCapacity", NC_DOUBLE)
        # max_capacity[:] = row[1].p_nom_max 

        id_ren += 1

## Add battery

In [ ]:
hydro_systems_i = n.storage_units_t.inflow.columns  # index of hydro systems (storage units with inflow)
batteries_i = n.storage_units.index.difference(hydro_systems_i)  # index of batteries (storage units without inflow)

hydro_systems = n.storage_units.loc[hydro_systems_i]
batteries = n.storage_units.loc[batteries_i]

id_batt = id_ren

if not batteries.empty:
    for (idx_name, row) in batteries.iterrows():

        tiub = master.createGroup(f"UnitBlock_{id_batt}")
        tiub.id = str(id_batt)
        tiub.type = "BatteryUnitBlock"
        

        # max power
        max_power = tiub.createVariable("MaxPower", NC_DOUBLE)
        max_power[:] = row.p_nom_opt * row.p_max_pu
        
        # min power
        min_power = tiub.createVariable("MinPower", NC_DOUBLE)
        min_power[:] = row.p_nom_opt * row.p_min_pu

        # max energy
        max_storage = tiub.createVariable("MaxStorage", NC_DOUBLE)
        max_storage[:] = row.p_nom_opt * row.max_hours

        # max energy
        min_storage = tiub.createVariable("MinStorage", NC_DOUBLE)
        min_storage[:] = 0.0

        # Initial storage
        initial_storage = tiub.createVariable("InitialStorage", NC_DOUBLE)
        initial_storage[:] = row.state_of_charge_initial * row.p_nom_opt * row.max_hours

        # Storing battery efficiency
        storing_efficiency = tiub.createVariable("StoringBatteryRho", NC_DOUBLE)
        storing_efficiency[:] = row.efficiency_store

        # Discharge battery efficiency
        storing_efficiency = tiub.createVariable("ExtractingBatteryRho", NC_DOUBLE)
        storing_efficiency[:] = row.efficiency_dispatch

        # # max capacity
        # max_capacity = tiub.createVariable("MaxCapacity", NC_DOUBLE)
        # max_capacity[:] = row[1].p_nom_max 

        id_batt += 1

## Add hydro block

In [ ]:
id_hydro = id_batt

if not hydro_systems.empty:
    for (idx_name, row) in hydro_systems.iterrows():

        tiub = master.createGroup(f"UnitBlock_{id_hydro}")
        tiub.id = str(id_hydro)
        tiub.type = "HydroUnitBlock"

        tiub.createDimension("NumberReservoirs", 1)  # optional, the number of reservoirs
        N_ARCS = 3
        tiub.createDimension("NumberArcs", N_ARCS)  # optional, the number of arcs connecting the reservoirs
        # No NumberIntervals
        
        MAX_FLOW = 100*n.storage_units_t.inflow.loc[:, idx_name].max()
        P_MAX = row.p_nom_opt * row.p_max_pu
        P_MIN = row.p_nom_opt * row.p_min_pu

        # StartArc
        start_arc = tiub.createVariable("StartArc", NC_UINT, ("NumberArcs",))
        start_arc[:] = np.full((N_ARCS,), 0, dtype=NP_UINT)

        # EndArc
        end_arc = tiub.createVariable("EndArc", NC_UINT, ("NumberArcs",))
        end_arc[:] = np.full((N_ARCS,), 1, dtype=NP_UINT)

        # MaxPower
        max_power = tiub.createVariable("MaxPower", NC_DOUBLE, ("NumberArcs",)) #, ("NumberArcs",)) #, ("TimeHorizon",)) #"NumberArcs"))
        max_power[:] = np.array([P_MAX, 0., 0.], dtype=NP_DOUBLE)

        # MinPower
        min_power = tiub.createVariable("MinPower", NC_DOUBLE, ("NumberArcs",)) #, ("NumberArcs",)) #, ("TimeHorizon",)) #"NumberArcs"))
        min_power[:] = np.array([0., 0., P_MIN], dtype=NP_DOUBLE)

        # MinFlow
        min_flow = tiub.createVariable("MinFlow", NC_DOUBLE, ("NumberArcs",)) #, ("TimeHorizon",))
        min_flow[:] = np.array([0., 0., -MAX_FLOW], dtype=NP_DOUBLE)

        # MaxFlow
        max_flow = tiub.createVariable("MaxFlow", NC_DOUBLE, ("NumberArcs",)) #, ("TimeHorizon",))
        max_flow[:] = np.array([P_MAX * 100., MAX_FLOW, 0.], dtype=NP_DOUBLE)
        
        # MinVolumetric
        min_volumetric = tiub.createVariable("MinVolumetric", NC_DOUBLE) #, ("TimeHorizon",))
        min_volumetric[:] = 0.0

        # MaxVolumetric
        max_volumetric = tiub.createVariable("MaxVolumetric", NC_DOUBLE)
        max_volumetric[:] = row.p_nom_opt * row.max_hours

        
        # Inflows
        inflows = tiub.createVariable("Inflows", NC_DOUBLE, ("NumberReservoirs", "TimeHorizon")) #,"NumberReservoirs",))  #"NumberReservoirs", 
        inflows[:] = np.array([n.storage_units_t.inflow.loc[:, idx_name]])

        # InitialVolumetric
        initial_volumetric = tiub.createVariable("InitialVolumetric", NC_DOUBLE) #, ("NumberReservoirs",))
        initial_volumetric[:] = row.state_of_charge_initial * row.max_hours * row.p_nom_opt

        # NumberPieces
        pieces = np.full((N_ARCS,), 1, dtype=NP_UINT)
        number_pieces = tiub.createVariable("NumberPieces", NC_UINT, ("NumberArcs",))
        number_pieces[:] = pieces

        # TotalNumberPieces
        tiub.createDimension("TotalNumberPieces", pieces.sum())

        # LinearTerm
        linear_term = tiub.createVariable("LinearTerm", NC_DOUBLE, ("TotalNumberPieces",))
        # linear_term[:] = np.array([1/n.storage_units.loc[idx_name, "efficiency_dispatch"], 0., n.storage_units.loc[idx_name, "efficiency_store"]], dtype=NP_DOUBLE)
        linear_term[:] = np.array([1/n.storage_units.loc[idx_name, "efficiency_dispatch"], 0., n.storage_units.loc[idx_name, "efficiency_store"]], dtype=NP_DOUBLE)

        # ConstTerm
        const_term = tiub.createVariable("ConstantTerm", NC_DOUBLE, ("TotalNumberPieces",))
        const_term[:] = np.full((N_ARCS,), 0.0, dtype=NP_DOUBLE)

In [ ]:
# id_hydro = id_batt

# if not hydro_systems.empty:
#     for (idx_name, row) in hydro_systems.iterrows():

#         tiub = master.createGroup(f"UnitBlock_{id_hydro}")
#         tiub.id = str(id_hydro)
#         tiub.type = "HydroUnitBlock"

#         # tiub.createDimension("NumberReservoirs", 1)  # optional, the number of reservoirs
#         # tiub.createDimension("NumberArcs", 1)  # optional, the number of arcs connecting the reservoirs
#         # First arc is for power flow, the second one is loss of water
#         tiub.createDimension("TotalNumberPieces", 1)  # optional, the number of arcs connecting the reservoirs

#         # # StartArc
#         # start_arc = tiub.createVariable("StartArc", NC_UINT)
#         # start_arc[:] = 0

#         # # EndArc
#         # end_arc = tiub.createVariable("EndArc", NC_UINT)
#         # end_arc[:] = np.array([1, 2], dtype=NP_UINT)

#         # MaxPower
#         max_power = tiub.createVariable("MaxPower", NC_DOUBLE) #, ("NumberArcs",)) #, ("TimeHorizon",)) #"NumberArcs"))
#         max_power[:] = row.p_nom_opt * row.p_max_pu

#         # MinFlow
#         min_flow = tiub.createVariable("MinFlow", NC_DOUBLE) #, ("TimeHorizon",))
#         min_flow[:] = 0.0

#         # MaxFlow
#         max_flow = tiub.createVariable("MaxFlow", NC_DOUBLE) #, ("TimeHorizon",))
#         max_flow[:] = row.p_nom_opt * row.p_max_pu
        
#         # MinVolumetric
#         min_volumetric = tiub.createVariable("MinVolumetric", NC_DOUBLE) #, ("TimeHorizon",))
#         min_volumetric[:] = 0.0

#         # MaxVolumetric
#         max_volumetric = tiub.createVariable("MaxVolumetric", NC_DOUBLE)
#         max_volumetric[:] = row.p_nom_opt * row.max_hours

#         # Inflows
#         inflows = tiub.createVariable("Inflows", NC_DOUBLE, ("TimeHorizon")) #,"NumberReservoirs",))  #"NumberReservoirs", 
#         inflows[:] = n.storage_units_t.inflow.loc[:, idx_name]

#         # InitialVolumetric
#         initial_volumetric = tiub.createVariable("InitialVolumetric", NC_DOUBLE) #, ("NumberReservoirs",))
#         initial_volumetric[:] = row.state_of_charge_initial * row.max_hours * row.p_nom_opt

#         # # FinalVolumetric
#         # finall_volume_reservoir = tiub.createVariable("FinalVolumetric", NC_DOUBLE) #, ("NumberReservoirs",))
#         # finall_volume_reservoir[:] = row.state_of_charge_initial * row.max_hours * row.p_nom_opt

#         # LinearTerm
#         linear_term = tiub.createVariable("LinearTerm", NC_DOUBLE)
#         linear_term[:] = 1/n.storage_units.loc[idx_name, "efficiency_dispatch"]

#         # ConstTerm
#         const_term = tiub.createVariable("ConstTerm", NC_DOUBLE)
#         const_term[:] = 0.0

In [ ]:
ds.close()

## Execute UCBlockSolver

In [ ]:
n.model.to_file("pypsa.lp")

In [ ]:
import subprocess
import os

PROJECT_PATH = "../../smspp-project"   # Path of the SMSpp project
COMPILE_MODE = "Release"              # Compilation mode (Debug/Release)

DATA_FOLDER = folder_builder  # Folder where all data inputs are contained (cwd will be moved here)
BSC_NAME = "uc_solverconfig.txt"          # Name of the file describing the BlockSolverConfig
UCFILE_NAME = "pypsa2smspp.nc4"      # Name of the UC-file to test

PARENT_ABSPATH_UCS = os.path.abspath(PROJECT_PATH + "/build/tools/ucblock_solver/" + COMPILE_MODE)
UCS_ABSPATH = os.path.abspath(PARENT_ABSPATH_UCS + "/ucblock_solver.exe")

result = subprocess.run(
    [UCS_ABSPATH, UCFILE_NAME, "-S", BSC_NAME],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    cwd=DATA_FOLDER,
)
result_ascii = result.stdout.decode('ascii')
print(result_ascii)

In [ ]:
res = re.search("Upper bound = (.*)\r\n", result_ascii)
smspp_obj = float(res.group(1))
print("SMS++ obj         : %.6f" % smspp_obj)
print("PyPSA dispatch obj: %.6f" % marg_cost.sum())
print("Error SMS++ - PyPSA dispatch [%%]: %.5f" % (100*(smspp_obj - marg_cost.sum())/marg_cost.sum()))

In [ ]:
marg_cost

In [ ]:
marg_cost.sum()

In [ ]:
a = sum([      1.51767897e+01,      1.74297319e+01,      1.54375597e+01,      1.62193759e+01,      1.30294407e+01 ])
a

In [ ]:
b = sum([     0.00000000000000e+00,     0.00000000000000e+00,     7.34239968739590e+00,     1.62212288564177e+01,     0.00000000000000e+00 ])
b

In [ ]:
a+b

In [ ]:
n.storage_units_t.p.sum()

In [ ]:
marg_cost

In [ ]:
n.storage_units.max_hours * n.storage_units.p_nom_opt

In [ ]:
n.storage_units_t.inflow

In [ ]:
marg_cost